In [1]:
from helpers.utilities import *
%run helpers/notebook_setup.ipynb

In [2]:
uniprot_to_entrez_path = 'data/uniprot_to_gene_id.tsv'
aptamers_path = 'data/other/relevant_aptamers.csv'
raw_protein_levels_path = 'data/clean/protein/indexed_by_target.csv'

# output
raw_gene_path = 'data/clean/protein/gene_levels_by_entrez.csv'
target_to_entrez_path = 'data/clean/protein/protein_to_entrez.csv'

In [3]:
relevant_aptamers = read_csv(aptamers_path, index_col=0)
uniprot_to_entrez = read_table(uniprot_to_entrez_path, names=['versioned_uniprot', 'entrez'])

In [4]:
raw_protein_matrix = read_csv(raw_protein_levels_path, index_col=0)

In [5]:
id_columns = ['Target', 'TargetFullName', 'UniProt', 'EntrezGeneID']

### Some aptamers are missing Entrez IDs.

In [6]:
relevant_aptamers[relevant_aptamers.EntrezGeneID.isnull()][id_columns]

,Target,TargetFullName,UniProt,EntrezGeneID
45,EGFRvIII,Epidermal growth factor receptor variant III,P00533,NaN
60,GI24,Platelet receptor Gi24,Q9H7M9,NaN
66,H2A3,Histone H2A type 3,Q7L7L0,NaN
71,HHLA2,HERV-H LTR-associating protein 2,Q9UM44,NaN
81,IFNA7,Interferon alpha-7,P01567,NaN
...,...,...,...,...
193,GDF-11/8,Growth/differentiation factor 11/8,O95390 O14793,NaN
310,C34 gp41 HIV Fragment,"gp41 C34 peptide, HIV",Q70626,NaN
620,Gro-b/g,Gro-beta/gamma,P19876 P19875,NaN
646,HSP 90a/b,Heat shock protein HSP 90-alpha/beta,P07900 P08238,NaN


### Updating Entrez IDs using a recent UniProt release

The aptamers metadata provide use with entrez ids for most of the aptamers. To fill in the gaps and settle for one id where multiple ids are provided I will map the proteins Entrez using the latest UniProt release: 

In [7]:
# just in case:
uniprot_to_entrez['uniprot'] = uniprot_to_entrez.versioned_uniprot.str.split('-').str[0]

one_protein_multiple_genes = uniprot_to_entrez[uniprot_to_entrez.uniprot.duplicated(keep=False)]
one_gene_multiple_proteins = uniprot_to_entrez[uniprot_to_entrez.entrez.duplicated(keep=False)]

The UniProt-Entrez mappings are in many-to-many relationship (as there may be multiple genes coding for specific protein and there may be multiple proteins coded by a single gene):

In [8]:
one_gene_multiple_proteins.sort_values('entrez')

,versioned_uniprot,entrez,uniprot
78,P04217,1,P04217
26655,V9HWD8,1,V9HWD8
1106,P18440,9,P18440
19348,Q400J6,9,Q400J6
26938,F5H5R8,9,F5H5R8
...,...,...,...
30372,B4DXM8,110117499,B4DXM8
4776,P0DPD6,110599583,P0DPD6
4838,P0DPD8,110599583,P0DPD8
165,Q9UG63,114483834,Q9UG63


In [9]:
one_protein_multiple_genes.sort_values('uniprot')

,versioned_uniprot,entrez,uniprot
32789,A0A024QZX0,107080638,A0A024QZX0
32790,A0A024QZX0,51256,A0A024QZX0
24652,A0A024R018,100287261,A0A024R018
24653,A0A024R018,100287502,A0A024R018
21533,A0A024R2A8,728239,A0A024R2A8
...,...,...,...
24049,V9HWE4,84220,V9HWE4
30038,X5D7D2,102724770,X5D7D2
30039,X5D7D2,8214,X5D7D2
22934,X5D7G6,51463,X5D7G6


In [10]:
uniprot_to_entrez

,versioned_uniprot,entrez,uniprot
0,P31946,7529,P31946
1,P62258,7531,P62258
2,Q04917,7533,Q04917
3,P61981,7532,P61981
4,P31947,2810,P31947
...,...,...,...
33561,A0A2R8YDF9,100996631,A0A2R8YDF9
33562,A0A0G2JM46,102725035,A0A0G2JM46
33563,A0A1B0GUV8,102723971,A0A1B0GUV8
33564,A0A0G2JMZ3,102725035,A0A0G2JMZ3


In [11]:
relevant_aptamers['EntrezFromUniprot'] = relevant_aptamers.UniProt.apply(
    lambda uniprot_ids: {
        entrez_id
        for uniprot_id in uniprot_ids.split(' ')
        for entrez_id in uniprot_to_entrez[uniprot_to_entrez.uniprot == uniprot_id].entrez
    }
)

In [12]:
relevant_aptamers[relevant_aptamers.EntrezFromUniprot == set()][id_columns + ['EntrezFromUniprot']]

,Target,TargetFullName,UniProt,EntrezGeneID,EntrezFromUniprot
91,IgA,Immunoglobulin A,P01876 P01877,3493 3494,{}
310,C34 gp41 HIV Fragment,"gp41 C34 peptide, HIV",Q70626,NaN,{}
633,HIV-2 Rev,Protein Rev_HV2BE,P18093,1724716,{}
640,HPV E7 Type 16,Protein E7_HPV16,P03129,1489079,{}
641,HPV E7 Type18,Protein E7_HPV18,P06788,1489089,{}
728,ILT-4,Leukocyte immunoglobulin-like receptor subfami...,Q8N423,10288,{}
736,IgD,Immunoglobulin D,P01880,3495 50802 3535,{}
737,IgE,Immunoglobulin E,P01854,3497 50802 3535,{}
738,IgM,Immunoglobulin M,P01871,3507 3512 50802 3535,{}
1314,IgG,Immunoglobulin G,P01857,3500 3501 3502 3503 50802 3535,{}


I did expect to find that the viral proteins do not map, but the IgG is worrying. Let's investigate:

In [13]:
# !cat data/msigdb/c2.cp.reactome.v6.2.symbols.gmt | grep -P "\tIG(A|D|E|M|G)"

I manually verified this and they are not mapped to NCBI counterparts. Maybe it is due to some sequence differences?

Interestingly, there are mappings in the aptamers file.

Note: There are no IG* (except for IGF*) genes in the Reactome GMT anyway (possible due to the file preparation, not because these do not appear in Reactome pathways!)

In [14]:
from numpy import nan

In [15]:
nan != nan

True

In [16]:
entrez = relevant_aptamers.EntrezGeneID.str.split(' ')

relevant_aptamers['AllEntrezIDs'] = relevant_aptamers.apply(lambda aptamer: (
    set(
        int(e)
        for e in [
            *(
                str(aptamer.EntrezGeneID).split(' ')
                if aptamer.EntrezGeneID else
                []
            ),
            *aptamer.EntrezFromUniprot
        ]
        if e != 'nan'
    )
), axis=1)

In [17]:
has_unambigious_mapping = relevant_aptamers.AllEntrezIDs.apply(len) == 1

simple_cases = relevant_aptamers[has_unambigious_mapping]
complicated_cases = relevant_aptamers[~has_unambigious_mapping]

In [18]:
len(simple_cases), len(complicated_cases)

(1240, 65)

In [19]:
len(complicated_cases) / len(relevant_aptamers) * 100

4.980842911877394

### Analysis of the protein-gene pairs that map one-to-one

In [20]:
handy_columns_subset = ['AptamerId', 'TargetFullName', 'UniProt', 'EntrezGeneSymbol', 'EntrezGeneID', 'EntrezFromUniprot', 'ConsensusEntrez']

Are there any duplicates?

In [21]:
pd.options.mode.chained_assignment = None

In [22]:
simple_cases['ConsensusEntrez'] = simple_cases.AllEntrezIDs.apply(lambda x: list(x)[0])

In [23]:
value_added = simple_cases[
    (~simple_cases.ConsensusEntrez.isnull()) & simple_cases.EntrezGeneID.isnull()
].ConsensusEntrez

In [24]:
simple_cases[
    simple_cases.ConsensusEntrez.isin(value_added)
][handy_columns_subset]

,AptamerId,TargetFullName,UniProt,EntrezGeneSymbol,EntrezGeneID,EntrezFromUniprot,ConsensusEntrez
45,5328-33,Epidermal growth factor receptor variant III,P00533,EGFR,NaN,{1956},1956
60,14123-34,Platelet receptor Gi24,Q9H7M9,C10orf54,NaN,{64115},64115
66,14144-3,Histone H2A type 3,Q7L7L0,HIST3H2A,NaN,{92815},92815
71,14132-21,HERV-H LTR-associating protein 2,Q9UM44,HHLA2,NaN,{11148},11148
81,14129-1,Interferon alpha-7,P01567,IFNA7,NaN,{3444},3444
97,10990-21,Leucine-rich repeat serine/threonine-protein k...,Q5S007,LRRK2,NaN,{120892},120892
139,14135-3,Relaxin receptor 1,Q9HBX9,RXFP1,NaN,{59350},59350
499,2677-1,Epidermal growth factor receptor,P00533,EGFR,1956,{1956},1956
1194,2879-9,Alpha-1-antichymotrypsin,P01011,SERPINA3,12,{12},12
1199,4153-11,Alpha-1-antichymotrypsin complex,P01011,SERPINA3,NaN,{12},12


This demonstrates the merit and shortcomings of filling in the empty IDS:
 - Epidermal growth factor receptor variant III is indeed produced by EGFR.
 - Alpha-1-antichymotrypsin complex tells us about SERPINA3 abundance.
 
However the rare isoforms may mislead in the enrichment analysis (we may not know of some function of the rare isoform, thus the gene set datasets / pathways may be biased).

We need to be careful here and verify if there is not "is subset of" relationship between these pairs:

In [25]:
simple_cases[
    simple_cases.ConsensusEntrez.isin(value_added)
    &
    simple_cases.ConsensusEntrez.duplicated(keep=False)
][handy_columns_subset]

,AptamerId,TargetFullName,UniProt,EntrezGeneSymbol,EntrezGeneID,EntrezFromUniprot,ConsensusEntrez
45,5328-33,Epidermal growth factor receptor variant III,P00533,EGFR,NaN,{1956},1956
499,2677-1,Epidermal growth factor receptor,P00533,EGFR,1956,{1956},1956
1194,2879-9,Alpha-1-antichymotrypsin,P01011,SERPINA3,12,{12},12
1199,4153-11,Alpha-1-antichymotrypsin complex,P01011,SERPINA3,NaN,{12},12


We would suspect the "is subset of" relationship when:
 - there is a correlation between the two (it does not have to be high: if one is much more abundant than the other, the correlation will fade; also some correlation will arise from the fact that they are coded by the same gene) and
 - one does never exceed the other

In [26]:
from scipy.stats import spearmanr

In [27]:
spearmanr(raw_protein_matrix.loc['ERBB1'], raw_protein_matrix.loc['EGFRvIII'])

SpearmanrResult(correlation=0.2785364043614303, pvalue=0.011280344040464886)

In [28]:
(raw_protein_matrix.loc['ERBB1'] > raw_protein_matrix.loc['EGFRvIII']).all()

True

In [29]:
spearmanr(raw_protein_matrix.loc['a1-Antichymotrypsin'], raw_protein_matrix.loc['alpha-1-antichymotrypsin complex'])

SpearmanrResult(correlation=0.10181648001218968, pvalue=0.36271624543319714)

In [30]:
(raw_protein_matrix.loc['a1-Antichymotrypsin'] > raw_protein_matrix.loc['alpha-1-antichymotrypsin complex']).all()

True

**This is suspiciously close to the criteria that I stated above. Maybe leaving these proteins aside is not a bad idea after all?**

Well, this should not change much given how low (relatively) is the abundance of the variant and complex:

In [31]:
raw_protein_matrix.loc['EGFRvIII'].mean() / raw_protein_matrix.loc['ERBB1'].mean()

0.006329930755933892

In [32]:
raw_protein_matrix.loc['alpha-1-antichymotrypsin complex'].mean() / raw_protein_matrix.loc['a1-Antichymotrypsin'].mean()

0.2715043075293972

### A look at the many-to-one case

In [33]:
complicated_cases['ConsensusEntrez'] = complicated_cases.AllEntrezIDs.apply(lambda x: ', '.join(map(str, sorted(x))))

In [34]:
show_table(complicated_cases[handy_columns_subset].sort_values('ConsensusEntrez'))

,AptamerId,TargetFullName,UniProt,EntrezGeneSymbol,EntrezGeneID,EntrezFromUniprot,ConsensusEntrez
310,4792-51,"gp41 C34 peptide, HIV",Q70626,Human-virus,NaN,{},
713,2829-19,Interleukin-27,Q8NEV9 Q14213,IL27 EBI3,246778 10148,"{246778, 10148}","10148, 246778"
353,3358-51,Cyclin-dependent kinase 5:Cyclin-dependent kinase 5 activator 1 complex,Q00535 Q15078,CDK5 CDK5R1,1020 8851,"{8851, 1020}","1020, 8851"
626,4914-10,Human Chorionic Gonadotropin,P01215 P01233,CGA CGB,1081 1082,{1081},"1081, 1082"
558,3032-11,Follicle stimulating hormone,P01215 P01225,CGA FSHB,1081 2488,"{2488, 1081}","1081, 2488"
799,2953-31,Luteinizing hormone,P01215 P01229,CGA LHB,1081 3972,"{1081, 3972}","1081, 3972"
1133,3521-16,Thyroid Stimulating Hormone,P01215 P01222,CGA TSHB,1081 12372,"{1081, 7252}","1081, 7252, 12372"
362,3714-49,Creatine kinase M-type:Creatine kinase B-type heterodimer,P12277 P06732,CKB CKM,1152 1158,"{1152, 1158}","1152, 1158"
36,13103-125,Chorionic somatomammotropin hormone,P0DML2 P0DML3,CSH1 CSH2,1442 1443,"{1442, 1443}","1442, 1443"
364,5225-50,Casein kinase II 2-alpha:2-beta heterotetramer,P68400 P67870,CSNK2A1 CSNK2B,1457 1460,"{1457, 1460}","1457, 1460"


### Analysis of all duplicates

In [35]:
from collections import Counter
number_of_proteins_mapping_to = Counter()

In [36]:
for entrez_ids in relevant_aptamers.AllEntrezIDs:
    for entrez in entrez_ids:
        number_of_proteins_mapping_to[entrez] += 1

In [37]:
duplicated = {
    entrez
    for entrez, count in number_of_proteins_mapping_to.items()
    if count > 1
}

In [38]:
len(duplicated), len(duplicated) / len(number_of_proteins_mapping_to) * 100

(52, 3.9068369646882046)

In [39]:
relevant_aptamers['ConsensusEntrez'] = relevant_aptamers.AllEntrezIDs.apply(lambda x: ', '.join(map(str, sorted(x))))

show_table(relevant_aptamers[relevant_aptamers.AllEntrezIDs.apply(
    lambda x: bool(x.intersection(duplicated)))
][handy_columns_subset].sort_values('ConsensusEntrez'))

,AptamerId,TargetFullName,UniProt,EntrezGeneSymbol,EntrezGeneID,EntrezFromUniprot,ConsensusEntrez
58,12060-28,Growth/differentiation factor 11,O95390,GDF11,10220,{10220},10220
626,4914-10,Human Chorionic Gonadotropin,P01215 P01233,CGA CGB,1081 1082,{1081},"1081, 1082"
558,3032-11,Follicle stimulating hormone,P01215 P01225,CGA FSHB,1081 2488,"{2488, 1081}","1081, 2488"
799,2953-31,Luteinizing hormone,P01215 P01229,CGA LHB,1081 3972,"{1081, 3972}","1081, 3972"
1133,3521-16,Thyroid Stimulating Hormone,P01215 P01222,CGA TSHB,1081 12372,"{1081, 7252}","1081, 7252, 12372"
5,7625-27,14-3-3 protein theta,P27348,YWHAQ,10971,{10971},10971
361,3800-71,Creatine kinase B-type,P12277,CKB,1152,{1152},1152
362,3714-49,Creatine kinase M-type:Creatine kinase B-type heterodimer,P12277 P06732,CKB CKM,1152 1158,"{1152, 1158}","1152, 1158"
363,2670-67,Creatine kinase M-type,P06732,CKM,1158,{1158},1158
1333,8446-4,Pituitary adenylate cyclase-activating polypeptide 27,P18509,ADCYAP1,116,{116},116


So we get lot's of things in the duplicates:
- fragments e.g. *Fibronectin Fragment 3*, *Fibronectin Fragment 4*;
- isoforms, e.g. *Apolipoprotein E (isoform E2)*, *Apolipoprotein E (isoform E3)*;
- variants (may be isoforms, modified or mutated proteins, not sure yet), e.g. *Epidermal growth factor receptor variant III*;
- mutations e.g. *Ubiquitin+1, truncated mutation for UbB*
- states (activated/inactivated), e.g. *Complement C3b*, *Complement C3b, inactivated*
- chains shared across multiple hormones, e.g. *Follicle stimulating hormone*, *Human Chorionic Gonadotropin*

CGA gene is an interesting case. This codes for an alpha chain of multiple glycoprotein hormones:

In [40]:
cga_aptamers = relevant_aptamers[relevant_aptamers.UniProt.str.contains('P01215')][handy_columns_subset + ['Target']]
cga_aptamers

,AptamerId,TargetFullName,UniProt,EntrezGeneSymbol,EntrezGeneID,EntrezFromUniprot,ConsensusEntrez,Target
558,3032-11,Follicle stimulating hormone,P01215 P01225,CGA FSHB,1081 2488,"{2488, 1081}","1081, 2488",FSH
626,4914-10,Human Chorionic Gonadotropin,P01215 P01233,CGA CGB,1081 1082,{1081},"1081, 1082",HCG
799,2953-31,Luteinizing hormone,P01215 P01229,CGA LHB,1081 3972,"{1081, 3972}","1081, 3972",Luteinizing hormone
1133,3521-16,Thyroid Stimulating Hormone,P01215 P01222,CGA TSHB,1081 12372,"{1081, 7252}","1081, 7252, 12372",TSH


I think that they have targeted the entire protein, and the hit indicates that both: the alpha chain and the remaining part of the protein is detected.

### Distributing the protein abundances across genes

Considerations:
  - I can distribute the values proportionally to the number of genes involved, which will probably give more weigh to some functional chains which are shared across many proteins
  - but can I have any better approximation?
  - one way would be only using the unique ones, but this would eliminate 10% of the data!

#### Create a reusable target -> entrez mapping:

In [41]:
def explode_dict(df, column: str):
    data = []
    for row in df.itertuples(index=False):
        base = row._asdict()
        for entry in base.pop(column):
            data.append({column: entry, **base})
    res = pd.DataFrame(data)
    return res

In [42]:
target_to_entrez = explode_dict(relevant_aptamers[['Target', 'AllEntrezIDs']], 'AllEntrezIDs')
target_to_entrez.tail(n=10)

,AllEntrezIDs,Target
1391,7124,TNF-a
1392,407977,TWEAK
1393,8742,TWEAK
1394,4803,b-NGF
1395,2260,bFGF-R
1396,1984,eIF-5A-1
1397,3674,gpIIbIIIa
1398,3690,gpIIbIIIa
1399,3384,sICAM-2
1400,4137,tau


In [43]:
target_to_entrez.to_csv(target_to_entrez_path)

#### Apply the mapping to the raw dataset:

In [44]:
from collections import defaultdict
raw_gene_matrix = defaultdict(lambda: Series(0, index=raw_protein_matrix.columns))

for target, row in raw_protein_matrix.iterrows():
    entrez_ids = target_to_entrez[target_to_entrez.Target == target].AllEntrezIDs
    for entrez_id in entrez_ids:
        raw_gene_matrix[entrez_id] += row / len(entrez_ids)
raw_gene_matrix = DataFrame(raw_gene_matrix).T

In [45]:
raw_gene_matrix

,149.TMD,007.TMD,001.TMD,064.TMD,151.TMD,...,177.HC,189.HC,217.HC,221.HC,245.HC
10273,174.1,521.1,57.0,158.1,62.1,...,12.6,11.4,12.2,15.8,22.2
1051,53.3,283.6,36.1,126.6,33.7,...,10.4,9.7,9.6,11.0,15.1
2026,11621.6,26883.3,10123.9,12015.4,3734.6,...,7699.7,5601.8,6421.7,8314.3,16016.8
51588,50.9,177.8,31.6,75.3,25.5,...,11.0,10.7,9.6,10.8,17.0
3587,174.6,385.3,116.8,192.7,104.0,...,50.2,46.9,44.9,53.3,82.9
...,...,...,...,...,...,...,...,...,...,...,...
5176,185698.9,199804.1,193171.3,208650.7,215160.2,...,177535.0,192301.2,206157.5,226385.4,159321.2
8722,951.8,2055.0,1290.1,1958.2,914.1,...,1535.7,1198.8,970.8,1465.3,2022.2
10841,52.1,161.3,27.3,70.2,20.8,...,21.2,19.7,18.6,18.4,21.2
29761,85.0,282.3,42.0,127.0,44.5,...,14.7,14.1,13.4,14.9,18.3


In [46]:
raw_protein_matrix

,149.TMD,007.TMD,001.TMD,064.TMD,151.TMD,...,177.HC,189.HC,217.HC,221.HC,245.HC
target,,,,,,,,,,,
CHIP,174.1,521.1,57.0,158.1,62.1,...,12.6,11.4,12.2,15.8,22.2
CEBPB,53.3,283.6,36.1,126.6,33.7,...,10.4,9.7,9.6,11.0,15.1
NSE,11621.6,26883.3,10123.9,12015.4,3734.6,...,7699.7,5601.8,6421.7,8314.3,16016.8
PIAS4,50.9,177.8,31.6,75.3,25.5,...,11.0,10.7,9.6,10.8,17.0
IL-10 Ra,174.6,385.3,116.8,192.7,104.0,...,50.2,46.9,44.9,53.3,82.9
...,...,...,...,...,...,...,...,...,...,...,...
PEDF,185698.9,199804.1,193171.3,208650.7,215160.2,...,177535.0,192301.2,206157.5,226385.4,159321.2
CATF,951.8,2055.0,1290.1,1958.2,914.1,...,1535.7,1198.8,970.8,1465.3,2022.2
FTCD,52.1,161.3,27.3,70.2,20.8,...,21.2,19.7,18.6,18.4,21.2


At least some rows have to differ:

In [47]:
assert (
    any(
        all(
            (gene_row != protein_row).all()
            for _, protein_row in raw_protein_matrix.iterrows()
        )
    )
    for gene_id, gene_row in raw_gene_matrix.iterrows()
)

In [48]:
raw_gene_matrix.to_csv(raw_gene_path)

**Note**: distributing the abundances using raw data and then transforming is different from transforming and then distributing the abundances!